# Redes neuronales artificiales

Durante las últimas clases haremos una exploración sobre varias técnicas de aprendizaje profundo (deep learning) comenzando con las redes neuronales artificiales (ANN), en particular las redes neuronales Feedforward. 

Usaremos el framework Keras, que es una API de alto nivel además de Tensorflow. Keras se está volviendo muy popular recientemente debido a su simplicidad. Es muy fácil crear modelos complejos e iterar rápidamente. 

## Introducción

Las redes neuronales artificiales (ANN) son redes neuronales conformadas por varias capas de neuronas totalmente conectadas:

![img1](https://raw.githubusercontent.com/Izainea/visualizacion/master/img/ANN1.png)


Consisten en una capa de entrada, varias capas ocultas y una capa de salida. Cada nodo de una capa está conectado a todos los demás nodos de la siguiente capa. Representa lo que estudiamos hace un momento, el perceptrón:

![img2](https://raw.githubusercontent.com/Izainea/visualizacion/master/img/ANN2.png)

Después de que cada perceptrón aplica los pesos obtenidos y se tiene la salida después de aplicar la función de activación entonces cada salida se convierte en la entrada para la siguiente capa. Los cálculos fluyen el diagrama de izquierda a derecha y la salida final se calcula realizando este procedimiento para todos los nodos. 

El objetivo de esta red neuronal profunda es aprender los pesos asociados a cada flecha de la primera gráfica, en otras palabras, consiste en la estimación de las siguientes matrices:

![img3](https://raw.githubusercontent.com/Izainea/visualizacion/master/img/ANN3.png)

En ese sentido, aprovechando el gráfico anterior, entendemos que la salida, para la red representada en esa figura, se calcula de la siguiente forma:

$$y=f(f(f(x\cdot W_1)\cdot W_2)\cdot W_3)$$

En este caso, todos los perceptrones tienen la misma función de activación $f$. El sesgo no esta incluido en la fórmula anterior, pero podemos ignorarlo mientras concebimos la intuición detrás de estas redes neuronales.

## Backpropagation 
Hasta ahora hemos descrito el cálculo de la salida a partir de unos pesos estimados pero ¿Cómo estimamos estos pesos?, en otras palabras, ¿Cómo entrenamos esta red? 

El siguiente algoritmo describe este proceso:

* Inicialice aleatoriamente los pesos de todos los nodos. 

* La salida final es el valor del último nodo. Compare la salida final con el objetivo real en los datos de entrenamiento y mida el error usando una función de pérdida (Loss function).

* Realice un pase hacia atrás de derecha a izquierda y propague el error a cada nodo individual utilizando la propagación hacia atrás. * Calcule la contribución de cada peso al error y ajuste los pesos en consecuencia utilizando el descenso de gradiente. Propague los gradientes de error a partir de la última capa.

La propagación hacia atrás con descenso de gradiente es el concepto fundamental detrás del entrenamiento de la red, en esencia buscamos minimizar el error y en ese sentido buscamos un valor mínimo de la función de pérdida. Los detalles matemáticos los omitiremos de esta explicación pero se pueden encontrar en este video:
[![imgvid](https://img.youtube.com/vi/9OzLcgy1bjs/0.jpg)](https://www.youtube.com/watch?v=9OzLcgy1bjs)



## ¿Por qué funcionan las redes?

La esencia de las redes neuronales, por lo menos en esta versión inicial, consiste en la posibilidad de proyectar-transformar los registros de entrada en un espacio con mayor dimensión, con eso el proceso de clasificación se hace más sencillo, el siguiente gráfico ilustra esta situación:

![img4](https://raw.githubusercontent.com/Izainea/visualizacion/master/img/ANN4.png)

La proyección a otra dimensión permitió que hicieramos una separación como la siguiente:

![img5](https://raw.githubusercontent.com/Izainea/visualizacion/master/img/ANN5.png)

En resumen, las ANN son modelos de aprendizaje profundo muy flexibles pero potentes, permiten estimar aproximaciones a cualquier función compleja. Su aumento de popularidad se ha debido a tres razones: trucos inteligentes que hicieron posible el entrenamiento de estos modelos, un gran aumento en la potencia computacional, especialmente GPU y entrenamiento distribuido, además de una gran cantidad de datos de entrenamiento.





## Implementación en Python

Vamos ahora a usar el framework `keras` para hacer algunas  clasificaciones. Iniciemos con lo que necesitamos para poder iniciar el desarrollo de los modelos:

In [0]:
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
import seaborn as sns
import warnings

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)


from datetime import datetime
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_classification, make_moons, make_circles
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
import keras.backend as K
from keras.wrappers.scikit_learn import KerasClassifier

In [0]:
!pip install keras

`make_classification` es una función que nos permite simular un data set para clasificar:

In [0]:
X, y = make_classification(n_samples=1000, n_features=2, n_redundant=0, 
                           n_informative=2, random_state=2020, n_clusters_per_class=1)
sns.scatterplot(x=X.T[0],y=X.T[1],hue=y)

Antes de usar `keras` veamos un clasificador lineal, la regresión logística, compararemos después como clasifica una red neuronal:

In [0]:
lr = LogisticRegression(solver='liblinear')
lr.fit(X, y)
y_pred=lr.predict(X)
sns.heatmap(confusion_matrix(y,y_pred),annot=True,fmt='.0f')

In [0]:
sns.scatterplot(x=X.T[0],y=X.T[1],hue=y,style=y_pred)
limits = np.array([-4, 4])
boundary = -(lr.coef_[0][0] * limits + lr.intercept_[0]) / lr.coef_[0][1]
plt.plot(limits, boundary, "g-", linewidth=2)

Ahora veamos que ocurre con `keras`, despleguemos el modelo de clasificación usando la sintaxis de keras, la documentación se puede encontrar aquí:
[Documentación Keras](https://keras.io/)

In [0]:
model = Sequential()
model.add(Dense(units=1, input_shape=(2,), activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x=X, y=y, verbose=1, epochs=100)

In [0]:
model.summary()

In [0]:
y_pred=model.predict_classes(X)

In [0]:
y_pred=(model.predict(X) > 0.5).astype("int32")
sns.heatmap(confusion_matrix(y,y_pred),annot=True,fmt='.0f')

In [0]:
history.history

In [0]:
def plot_loss_accuracy(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, max(1, historydf.values.max())))
    loss = history.history['loss'][-1]
    acc = history.history['accuracy'][-1]
    plt.title('Loss: %.3f, Accuracy: %.3f' % (loss, acc))
plot_loss_accuracy(history)

In [0]:
model = Sequential()
model.add(Dense(units=4, input_shape=(2,), activation='sigmoid'))
model.add(Dense(units=4,  activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x=X, y=y, verbose=0, epochs=100)


In [0]:
model.summary()

In [0]:
y_pred=(model.predict(X) > 0.5).astype("int32")
sns.heatmap(confusion_matrix(y,y_pred),annot=True,fmt='.0f')

In [0]:
plot_loss_accuracy(history)

In [0]:
sns.scatterplot(x=X.T[0],y=X.T[1],hue=y,style=y_pred.T[0])

Usaremos [El modelo secuencial](https://keras.io/guides/sequential_model/) de keras que nos permite construir redes neuronales profundas apilando capas una encima de otra.

En el primer ejemplo se construye un modelo de regresión logística simple, todas las entradas están conectadas directamente al nodo de salida, sin capas ocultas. Es decir, calculamos $W$ para el modelo $y=f(x\cdot W)$, con $f$ la función sigmoidea.



En Keras no agregamos capas correspondientes a los nodos de entrada, solo lo hacemos para los nodos ocultos y de salida. En el primer modelo no tenemos capas ocultas, los nodos de entrada están conectados directamente al nodo de salida. Esto significa que nuestra definición de red neuronal en Keras solo tendrá una capa con un nodo, correspondiente al nodo de salida.

Mientras que en el segundo hacemos una red como la que definimos en el gráfico de ejemplo, aquí la función de activación en todas las capas es la función sigmoide.

***Algunas anotaciones importantes en la creación del modelo secuencial en keras***

La función Dense en Keras construye una capa de red neuronal completamente conectada, inicializando automáticamente los pesos como sesgos. Es una función muy útil que verá que se usa en todas partes. Los argumentos de la función se definen de la siguiente manera:

* **units**: El primer argumento, que representa el número de nodos en esta capa.

* **input_shape**: La primera capa en los modelos de Keras necesita especificar las dimensiones de entrada. Las capas posteriores no necesitan especificar este argumento.

* **activation**: La función de activación.

Posteriormente compilamos el modelo con la función `model.compile`. Esto crea el modelo de red neuronal especificando los detalles del proceso de aprendizaje. El modelo aún no se ha entrenado en este momento,  apenas declaramos el optimizador para usar y la función de pérdida para minimizar. Los argumentos para la función de compilación se definen de la siguiente manera:

* **optimizer**: Optimizador a usar para minimizar la función de pérdida.
* **loss**: La función de pérdida para minimizar.

* **metrics**: sobre qué métrica informar las estadísticas del modelo, para problemas de clasificación, lo configuramos como exactitud.

Finalmente compilamos con `model.history`, sus argumentos son:

* **x**: Los datos de entrada, los definimos como X arriba. Contiene las coordenadas $x$ e $y$ de los puntos de entrada.
* **y**: Se refiere a las etiquetas, en nuestro caso, la clase que estamos tratando de predecir: 0 o 1.
* **verbose**: Imprime la pérdida y la precisión, configúrelo en 1 para ver el resultado.

* **epochs**: Número de iteraciones para que la red repase todos los datos de entrenamiento. 

## Una visualización del clasificador

La siguiente función nos permite visualizar la forma en que clasifica el modelo, veamos:

In [0]:
def plot_decision_boundary(func, X, y, figsize=(9, 6)):
    amin, bmin = X.min(axis=0) - 0.1
    amax, bmax = X.max(axis=0) + 0.1
    hticks = np.linspace(amin, amax, 101)
    vticks = np.linspace(bmin, bmax, 101)
    
    aa, bb = np.meshgrid(hticks, vticks)
    ab = np.c_[aa.ravel(), bb.ravel()]
    c = func(ab)
    cc = c.reshape(aa.shape)

    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    
    fig, ax = plt.subplots(figsize=figsize)
    contour = plt.contourf(aa, bb, cc, cmap=cm, alpha=0.8)
    
    ax_c = fig.colorbar(contour)
    ax_c.set_label("$P(y = 1)$")
    ax_c.set_ticks([0, 0.25, 0.5, 0.75, 1])
    
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright)
    plt.xlim(amin, amax)
    plt.ylim(bmin, bmax)

In [0]:
plot_decision_boundary(lambda x: model.predict(x), X, y)

In [0]:
plot_decision_boundary(lambda x: lr.predict_proba(x).T[1], X, y)

## Datos no "linealmente separables"

Usemos otra función de `sklearn` para simular datos más dificiles de separar:

In [0]:
#make_moons
X, y = make_moons(n_samples=1000, noise=0.05, random_state=2020)
sns.scatterplot(x=X.T[0],y=X.T[1],hue=y)

In [0]:
model = Sequential()
model.add(Dense(units=4, input_shape=(2,), activation='tanh'))
model.add(Dense(units=4,  activation='tanh'))
model.add(Dense(units=4,  activation='tanh'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x=X, y=y, verbose=0, epochs=100)


In [0]:
model.summary()

In [0]:
y_pred=(model.predict(X) > 0.5).astype("int32")
sns.heatmap(confusion_matrix(y,y_pred),annot=True,fmt='.0f')

In [0]:
plot_loss_accuracy(history)

In [0]:
sns.scatterplot(x=X.T[0],y=X.T[1],hue=y,style=y_pred.T[0])

In [0]:
plot_decision_boundary(lambda x: model.predict(x), X, y)

In [0]:
#make_circles
X, y = make_circles(n_samples=1000, noise=0.02, random_state=2020)
sns.scatterplot(x=X.T[0],y=X.T[1],hue=y)

In [0]:
model = Sequential()
model.add(Dense(units=4, input_shape=(2,), activation='tanh'))
model.add(Dense(units=4,  activation='tanh'))
model.add(Dense(units=4,  activation='tanh'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x=X, y=y, verbose=0, epochs=100)


In [0]:
model.summary()

In [0]:
y_pred=(model.predict(X) > 0.5).astype("int32")
sns.heatmap(confusion_matrix(y,y_pred),annot=True,fmt='.0f')

In [0]:
plot_loss_accuracy(history)

In [0]:
sns.scatterplot(x=X.T[0],y=X.T[1],hue=y,style=y_pred.T[0])

In [0]:
plot_decision_boundary(lambda x: model.predict(x), X, y)

In [0]:
#make_multiclass
def make_multiclass(N=500, D=2, K=3):
    """
    N: number of points per class
    D: dimensionality
    K: number of classes
    """
    np.random.seed(0)
    X = np.zeros((N*K, D))
    y = np.zeros(N*K)
    for j in range(K):
        ix = range(N*j, N*(j+1))
        # radius
        r = np.linspace(0.0,1,N)
        # theta
        t = np.linspace(j*4,(j+1)*4,N) + np.random.randn(N)*0.2
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        y[ix] = j
    fig = plt.figure(figsize=(6, 6))
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu, alpha=0.8)
    plt.xlim([-1,1])
    plt.ylim([-1,1])
    return X, y
X, y = make_multiclass(K=3)
sns.scatterplot(x=X.T[0],y=X.T[1],hue=y)

In [0]:
y

In [0]:
model = Sequential()
model.add(Dense(128, input_shape=(2,), activation='tanh'))
model.add(Dense(64,  activation='tanh'))
model.add(Dense(32,  activation='tanh'))
model.add(Dense(3,  activation='softmax'))
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

y_cat = to_categorical(y)
history = model.fit(X, y_cat, verbose=0, epochs=20)
plot_loss_accuracy(history)

In [0]:
model.summary()

In [0]:
def plot_multiclass_decision_boundary(model, X, y):
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 101), np.linspace(y_min, y_max, 101))
    cmap = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

    Z = model.predict_classes(np.c_[xx.ravel(), yy.ravel()], verbose=0)
    Z = Z.reshape(xx.shape)
    fig = plt.figure(figsize=(8, 8))
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
plot_multiclass_decision_boundary(model, X, y)

In [0]:
plot_loss_accuracy(history)

In [0]:
sns.scatterplot(x=X.T[0],y=X.T[1],hue=y,style=y_pred.T[0])

**Ejercicio**

Implemente los tres modelos de clasificación expuestos en esta clase y concluya respecto a las soluciones obtenidas.